In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import numpy as np
import webbrowser
import os
from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

In [2]:
files = os.listdir()
flink= []
for file in files:
    if file.find('DS')!=-1:
        flink.append(file)
flink    

['DS all links.csv',
 'DS data.csv',
 'DS Fulltime links.csv',
 'DS intern links.csv',
 'DS Other links.csv',
 'DS Parttime links.csv']

In [3]:
df = pd.DataFrame()
for fl in flink:
    d1 = pd.read_csv(fl)
    df = pd.concat((df,d1))
df

,0,title,location,script,company,description,level,type,function,industry
0,https://www.linkedin.com/jobs/view/data-scient...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://www.linkedin.com/jobs/view/senior-data...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://www.linkedin.com/jobs/view/data-scient...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://www.linkedin.com/jobs/view/machine-lea...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://www.linkedin.com/jobs/view/data-scient...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
51,https://www.linkedin.com/jobs/view/ase-stem-le...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
52,https://www.linkedin.com/jobs/view/mls-medical...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53,https://www.linkedin.com/jobs/view/forensic-sc...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54,https://www.linkedin.com/jobs/view/part-time-i...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df.to_csv('DS all links.csv', index =False)

In [5]:
links = df['0'].values
len(links), links[0]

(535, 'https://www.linkedin.com/jobs/view/data-scientist-at-amazon-2454065128')

In [6]:
def get_page(url):
    driver = webdriver.Chrome('chromedriver',options=options)
    driver.get(url)
    return BeautifulSoup(driver.page_source, 'html.parser')

In [7]:
data = []
i = 0

while i < len(links):
    record = {}
    print(i,links[i])
    try:
        #webbrowser.open(link)
        time.sleep(np.abs(np.random.normal(0,1)))
        s0 = get_page(links[i])

        # title
        record['title'] = s0.find('h1').text

        # location
        record['location'] = s0.find('span',class_="sub-nav-cta__meta-text").text

        # script
        record['script']=s0.find('script',type="application/ld+json").contents[0]

        # Company
        record['company'] = s0.find('span',class_="topcard__flavor").text

        # Job description
        record['description']=s0.find('div', class_="description__text description__text--rich").text

        # Criteria
        criteria = s0.find_all('li',class_="job-criteria__item")

        # Seniority level
        record['level'] = criteria[0].find('span').text
        # Job type
        record['type'] = criteria[1].find('span').text
        # Job function
        record['function']= criteria[2].find('span').text
        # Industry
        inds = criteria[3].find_all('span')
        inds = [x.text for x in inds]
        record['industry']=', '.join(inds)
        i+=1
        time.sleep(np.abs(2+np.random.normal(0,1)))
        trial = 0
    except: 
        print('take a break 10sec')
        time.sleep(np.abs(15+np.random.normal(0,2)))
        trial+=1
        if trial > 5: i+=1
        continue
    print(record['title'],record['company'])
    data.append(record)

0 https://www.linkedin.com/jobs/view/data-scientist-at-amazon-2454065128
Data Scientist Amazon
1 https://www.linkedin.com/jobs/view/senior-data-scientist-at-github-2444405124
Senior Data Scientist GitHub
2 https://www.linkedin.com/jobs/view/data-scientist-at-facebook-2443202248
Data Scientist Facebook
3 https://www.linkedin.com/jobs/view/machine-learning-engineer-apple-pay-at-apple-2454522010
Machine Learning Engineer, Apple Pay Apple
4 https://www.linkedin.com/jobs/view/data-scientist-at-stem-it-2443264882
Data Scientist Stem IT
5 https://www.linkedin.com/jobs/view/data-scientist-commodityfx-analytics-at-mckinsey-company-2384553718
Data Scientist, CommodityFX Analytics McKinsey & Company
6 https://www.linkedin.com/jobs/view/data-scientist-insurance-at-tesla-2454545848
Data Scientist, Insurance Tesla
7 https://www.linkedin.com/jobs/view/associate-data-scientist-at-pfizer-2454543895
Associate Data Scientist Pfizer
8 https://www.linkedin.com/jobs/view/usa-data-scientist-at-walmart-245257

In [8]:
dat = pd.DataFrame(data)

In [9]:
dat.to_csv('DS data.csv', index=False)

In [10]:
import json

In [12]:
dat.script[0]

'{"@context":"http://schema.org","@type":"JobPosting","datePosted":"2021-03-17T15:24:00.000Z","description":"\\u003Cstrong\\u003E\\u003Cu\\u003EDescription\\u003Cbr\\u003E\\u003Cbr\\u003E\\u003C/u\\u003E\\u003C/strong\\u003EThe Amazon Devices-Demand Planning team is seeking an outstanding scientist with strong analytical and communication skills to help with demand forecasting and supply optimization for the entire Amazon device family of products and accessories. We develop scalable and robust state-of-the-art solutions that involve learning from different data sources. This role is central to the continued growth of Amazon Device division as we have grown from the first Kindle E-Reader to a vast portfolio of Echo, Fire TV, Fire Tablet, E-Reader, Ring and many other devices. With better forecasts we drive down supply chain costs, enabling the offer of lower prices and better in-stock selection for our customers.\\u003Cbr\\u003EIn this role, you will have an opportunity to both develop

In [17]:
res = json.loads(dat.script[1]) 
res

{'@context': 'http://schema.org',
 '@type': 'JobPosting',
 'datePosted': '2021-03-18T00:09:18.000Z',
 'description': "<br>GitHub helps companies, organizations, and groups of individuals succeed by allowing them to build better software, together. The engineering organization is looking for a Senior Data Scientist to join the Data Science group and closely partner with our Product organization to unlock insight and optimize our product’s surface area.\xa0<br> <br>You will work on building and maintaining data products and tools and executing on analyses that will help inform our broader company strategy and individual product area needs. You’ll get to work with large, complex data sets and solve difficult, non-routine problems, applying advanced methods as needed.\xa0<br> <br>This is an individual contributor position with significant growth potential.<br> <br><strong>Responsibilities<br></strong> <ul><li>Engage with Product and other embedded Data Scientists across the landscape that 

In [18]:
for key, val in res.items():
    print(key)

@context
@type
datePosted
description
employmentType
experienceRequirements
hiringOrganization
identifier
image
industry
jobLocation
skills
title
validThrough
